In [1]:
#import things to use
import os
import numpy as np
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Activation, Masking, Dense 
from keras.layers import Convolution2D as Conv2D 
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D 
from keras.layers import Softmax, ReLU, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn import metrics 
print('finished imports!')

Using TensorFlow backend.


finished imports!


In [2]:
#import training data
#see getLabels_stackedData3.py for preprocessing

#run for train data only
print(os.getcwd())
os.chdir('/home/ec2-user/SageMaker')
print(os.getcwd())
labels = []
allCols = []
allRows = []
allSTFTs = []
paddedSTFTs = []
maxTime = 0
print('made variables')

#open file (lists all test/train .npy STFT file names) 
file = open('Top12List.txt', 'r')
#file = open('trainList.txt', 'r')
data = file.read().split('\n')
file.close()
print('got data files in a list')
print(data)

os.chdir('./aws_pilotwhales2')
print('changed to aws_pilotwhales2 folder')

for array in data:
    #loop through to find out if file is a buzz or minibuzz, and add label accordingly 
    nameParse = array.split("_",-1)
    #print(nameParse)
    typeParse = nameParse[4].split("u", -1)
    #print(typeParse)
    if(typeParse[0] == 'b'):
        labels.append(1)
    elif(typeParse[0] == 'minib'):
        labels.append(0)
    else: 
        print('Error, not a buzz or minibuzz!')
    
    curSTFT = np.load(array)
        
    #find cols (number of time steps) of each STFT and save longest one
    rows, cols = curSTFT.shape
    allCols.append(cols)
    allRows.append(rows)
    if (cols>maxTime):
        maxTime = cols
    if (rows!=1025):
        print('Error, not 1025 STFT coefficients') 
        
    allSTFTs.append(curSTFT)
    
print(labels)
print(allRows)
print(allCols)
print(maxTime) 
print('first loop done')

for array in data: 
    #loop though STFTs again to zero pad and transpose  
    #NOTE: must be done after we definitvely know the max number of time steps
    curSTFT = np.load(array)
    rows, cols = curSTFT.shape
    pad = maxTime-cols
    
    zeroPad = np.zeros((rows,maxTime-cols))
    paddedSTFT = np.append(curSTFT, zeroPad, axis = 1)
    
    paddedSTFT = np.transpose(paddedSTFT)
    paddedSTFT = np.reshape(paddedSTFT, paddedSTFT.shape + (1,))
    
    paddedSTFTs.append(paddedSTFT)

print('second loop done')

labels = np.array(labels)
hotlabels = keras.utils.to_categorical(labels, num_classes=2, dtype='float32')
samples = np.array(paddedSTFTs)
print('hotlabels adn samples in np array')

/home/ec2-user/SageMaker
/home/ec2-user/SageMaker
made variables
got data files in a list
['STFT_F_gm266a_event732_buzz124.npy', 'STFT_F_gm185b_event178_minibuzz28.npy', 'STFT_F_gm266a_event1202_buzz493.npy', 'STFT_F_gm185b_event199_minibuzz43.npy', 'STFT_F_gm267a_event1031_buzz276.npy', 'STFT_F_gm185b_event249_minibuzz74.npy', 'STFT_F_gm266a_event1204_buzz495.npy', 'STFT_F_gm208a_event443_minibuzz14.npy', 'STFT_F_gm267a_event1521_buzz387.npy', 'STFT_F_gm187b_event1602_minibuzz91.npy', 'STFT_F_gm266a_event1136_buzz439.npy', 'STFT_F_gm187b_event815_minibuzz62.npy']
changed to aws_pilotwhales2 folder
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
[1025, 1025, 1025, 1025, 1025, 1025, 1025, 1025, 1025, 1025, 1025, 1025]
[559, 94, 784, 75, 1047, 90, 484, 158, 503, 90, 559, 109]
1047
first loop done
second loop done
hotlabels adn samples in np array


In [4]:
#delete


In [3]:
#layer 0: input
#labels[]
#paddedSTFTs[] (maxTime rows, 1025 cols) each

print('ready to start model')

# build model
model = Sequential()

print('made model')

#NOTE, CHANGED PADDING ON 2D CONVOLUTIONS from 'valid' to 'same'
#layer 1: 2D convolution between input and 256 filters with 1 row and 1025 cols
model.add(Conv2D(256, input_shape = [maxTime,1025,1], kernel_size = [1,1025], strides=(1, 1), padding='valid', data_format="channels_last", dilation_rate=(1, 1), activation=None, use_bias=False, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
#batch normalization- add in layer? don't understand parameters well
#model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
#reLU layer
model.add(ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
print('layer 1 done')

##layer 2: 2D convolution between output of layer 1 and 256 filters with 3 rows and 256 cols
model.add(Conv2D(256, kernel_size = [3,256], strides=(2, 1), padding='same', data_format="channels_last", dilation_rate=(1, 1), activation=None, use_bias=False, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
##batch normalization- add in layer? don't understand parameters well
##model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
##reLU layer
model.add(ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
print('layer 2 done')

#layer 3: 2D convolution between output of layer 2 and 256 filters with 3 rows and 256 cols
model.add(Conv2D(256, kernel_size = [3,256], strides=(2, 1), padding='same', data_format="channels_last", dilation_rate=(1, 1), activation=None, use_bias=False, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
##batch normalization- add in layer? don't understand parameters well
##model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
##reLU layer
model.add(ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
print('layer 3 done')

#layer 4: 2D convolution between output of layer 3 and 256 filters with 3 rows and 256 cols
model.add(Conv2D(256, kernel_size = [3,256], strides=(2, 1), padding='same', data_format="channels_last", dilation_rate=(1, 1), activation=None, use_bias=False, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
##batch normalization- add in layer? don't understand parameters well
##model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
##reLU layer
model.add(ReLU(max_value=None, negative_slope=0.0, threshold=0.0))
print('layer 4 done')

#layer 5: Global max pooling
model.add(GlobalMaxPooling2D(data_format="channels_last"))
print('layer 5 done')

#layer 6: fully connected layer
model.add(Dense(2, activation='softmax', use_bias=False, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None))
print('layer 6 done')

#Compile model [COMPILE]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
print('compiled')

#Now let us train our model [FIT]
ES = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=2, mode='auto', baseline=None, restore_best_weights=True)
#test
#model.fit(x=samples, y=hotlabels, batch_size=1, epochs=6, verbose=2, callbacks=[ES], validation_split=0.2, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
#actual, not changed #epochs from 26 -->1  
model.fit(x=samples, y=hotlabels, batch_size=1, epochs=12, verbose=2, callbacks=[ES], validation_split=0.2, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
print('ran fit')

#[EVALUATE]
#test
#model.evaluate(x=samples, y=hotlabels, batch_size=1, verbose=1, sample_weight=None, steps=None)
#actual
model.evaluate(x=samples, y=hotlabels, batch_size=1, verbose=1, sample_weight=None, steps=None)

#[PREDICT (w/TestData I kept aside)]
#preprocess STFT data in TestData folder!
#see getLabels_stackedData3.py, but don't give labels
##model.predict(testSamples, batch_size=26, verbose=1, steps=None, callbacks=ES)

ready to start model
made model
layer 1 done
layer 2 done
layer 3 done
layer 4 done
layer 5 done
layer 6 done
compiled
Train on 9 samples, validate on 3 samples
Epoch 1/12
 - 38s - loss: 0.6933 - acc: 0.5556 - val_loss: 0.6932 - val_acc: 0.3333
Epoch 2/12
 - 35s - loss: 0.6921 - acc: 0.5556 - val_loss: 0.6998 - val_acc: 0.3333
Epoch 3/12
 - 34s - loss: 0.6209 - acc: 0.5556 - val_loss: 1.3731 - val_acc: 0.3333
Epoch 4/12
 - 34s - loss: 0.7660 - acc: 0.5556 - val_loss: 4.8157 - val_acc: 0.3333
Epoch 5/12
 - 34s - loss: 0.6074 - acc: 0.6667 - val_loss: 0.6978 - val_acc: 0.3333
Restoring model weights from the end of the best epoch
Epoch 00005: early stopping
ran fit
12/12 [==============================] - 12s 1000ms/step


[0.6931138088305792, 0.5]